## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [1]:
import pandas as pd
import requests
import numpy as np

# 1) Meu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [2]:
# 2) Meu código aqui
print("\nNúmero de valores ausentes por variável:")
print(sinasc.isnull().sum())


Número de valores ausentes por variável:
ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64


In [3]:
# 3) Meu código aqui
colunas_de_interesse = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
                           'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']

    # Criando uma nova base de dados apenas com as colunas de interesse
sinasc_selecionado = sinasc[colunas_de_interesse].copy()

    # Refazendo a contagem de valores ausentes por variável
print("Número de valores ausentes por variável (após seleção de colunas):")
print(sinasc_selecionado.isnull().sum())

Número de valores ausentes por variável (após seleção de colunas):
LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64


In [4]:
# 4) Meu código aqui
# Removendo os registros com APGAR5 não preenchido
sinasc_selecionado = sinasc_selecionado.dropna(subset=['APGAR5']).copy()
numero_linhas = sinasc_selecionado.shape[0]
print(f"Número de linhas após remover registros com APGAR5 não preenchido: {numero_linhas}")
print("Número de valores ausentes por variável após remoção de registros com APGAR5 não preenchido:")
print(sinasc_selecionado.isnull().sum())

Número de linhas após remover registros com APGAR5 não preenchido: 26925
Número de valores ausentes por variável após remoção de registros com APGAR5 não preenchido:
LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64


In [5]:
# 5) seu código aqui
# Substitui os valores 9 por NaN (Not a Number) nas colunas 'ESTCIVMAE' e 'CONSULTAS'
sinasc_selecionado[['ESTCIVMAE', 'CONSULTAS']] = sinasc_selecionado[['ESTCIVMAE', 'CONSULTAS']].replace(9, np.nan)

print("Número de valores ausentes por variável após substituir o código 9 por NaN:")
print(sinasc_selecionado.isnull().sum())


Número de valores ausentes por variável após substituir o código 9 por NaN:
LOCNASC          0
IDADEMAE         0
ESTCIVMAE      459
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS       50
APGAR5           0
dtype: int64


In [6]:
# 6) Meu código aqui
# Substitui os valores faltantes em 'QTDFILVIVO' por zero
sinasc_selecionado['QTDFILVIVO'] = sinasc_selecionado['QTDFILVIVO'].fillna(0)


print("Número de valores ausentes por variável após substituir valores faltantes em QTDFILVIVO por zero:")
print(sinasc_selecionado.isnull().sum())

Número de valores ausentes por variável após substituir valores faltantes em QTDFILVIVO por zero:
LOCNASC          0
IDADEMAE         0
ESTCIVMAE      459
ESCMAE         310
QTDFILVIVO       0
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS       50
APGAR5           0
dtype: int64


In [7]:
# 7) Meu código aqui
# Preenche os valores faltantes restantes com 'Não Informado'
sinasc_selecionado = sinasc_selecionado.fillna('Não Informado')
    
print("Número de valores ausentes por variável após preencher todos os faltantes restantes:")
print(sinasc_selecionado.isnull().sum())

Número de valores ausentes por variável após preencher todos os faltantes restantes:
LOCNASC       0
IDADEMAE      0
ESTCIVMAE     0
ESCMAE        0
QTDFILVIVO    0
GESTACAO      0
GRAVIDEZ      0
CONSULTAS     0
APGAR5        0
dtype: int64


In [8]:
# 8) Meu código aqui
sinasc_selecionado = sinasc_selecionado.fillna('Não Informado')

    # Definindo a função para categorizar o Apgar
def categoriza_apgar(apgar):
    if 8 <= apgar <= 10:
         return 'normal'
    elif 6 <= apgar <= 7:
        return 'asfixia leve'
    elif 4 <= apgar <= 5:
        return 'asfixia moderada'
    elif 0 <= apgar <= 3:
        return 'asfixia severa'
    else:
        return 'Não Classificado'
    # Criando a nova coluna com a classificação do Apgar
sinasc_selecionado['apgar_cat'] = sinasc_selecionado['APGAR5'].apply(categoriza_apgar)


    # Calculando a frequência de cada categoria
frequencias = sinasc_selecionado['apgar_cat'].value_counts()

print("Frequência de cada categoria de Apgar:")
print(frequencias)

Frequência de cada categoria de Apgar:
apgar_cat
normal              26463
asfixia leve          320
asfixia severa         74
asfixia moderada       68
Name: count, dtype: int64


In [9]:
# 9) Meu código aqui
# Renomei as colunas para snake_case
sinasc_selecionado.columns = sinasc_selecionado.columns.str.lower().str.replace(' ', '_')


print("Nomes das colunas após a transformação para snake case:")
print(sinasc_selecionado.columns)

Nomes das colunas após a transformação para snake case:
Index(['locnasc', 'idademae', 'estcivmae', 'escmae', 'qtdfilvivo', 'gestacao',
       'gravidez', 'consultas', 'apgar5', 'apgar_cat'],
      dtype='object')
